In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn import preprocessing

In [2]:
# Step 1: Data Preprocessing and Label Encoding
def df_label_encoder(df, columns):
    le = preprocessing.LabelEncoder()
    for col in columns:
        df[col] = le.fit_transform(df[col].astype(str))
    return df

In [3]:
def preprocess(df):
    df = df_label_encoder(df, ['merchant', 'category', 'city', 'state', 'job'])
    df['amt'] = (df['amt'] - df['amt'].min()) / (df['amt'].max() - df['amt'].min())
    df['node_from'] = df['cc_num'].astype(str)
    df['node_to'] = df['merchant'].astype(str)
    df = df.sort_values(by=['node_from'])
    node_list = pd.concat([df['node_from'], df['node_to']]).unique()
    return df, node_list

In [4]:
def create_graph_data(df, node_list):
    node_map = {node: idx for idx, node in enumerate(node_list)}
    edge_index = np.array([
        [node_map[from_node], node_map[to_node]] for from_node, to_node in zip(df['node_from'], df['node_to'])
    ], dtype=np.int64).T
    node_features = np.array(df[['amt', 'category', 'city', 'state']].values, dtype=np.float32)
    labels = np.array(df['is_fraud'].values, dtype=np.int64)
    return node_features, edge_index, labels

In [5]:
# Load dataset and preprocess
df = pd.read_csv('creditcard/fraudTrain.csv')  # Update with your .csv file path

# Reduce the dataset to 1/5th (20%) of the original size
df = df.sample(frac=0.2, random_state=42)

df, node_list = preprocess(df)

In [6]:
# Create the graph dataset
node_features, edge_index, labels = create_graph_data(df, node_list)

In [7]:
# Step 2: Define the WGAN Generator and Discriminator (same as before)
class WGANGenerator(tf.keras.Model):
    def __init__(self, latent_dim, hidden_dim, output_dim):
        super(WGANGenerator, self).__init__()
        self.fc1 = tf.keras.layers.Dense(hidden_dim, activation='relu')
        self.fc2 = tf.keras.layers.Dense(output_dim)

    def call(self, z):
        x = self.fc1(z)
        return self.fc2(x)

In [8]:
class WGANDiscriminator(tf.keras.Model):
    def __init__(self, input_dim, hidden_dim):
        super(WGANDiscriminator, self).__init__()
        self.fc1 = tf.keras.layers.Dense(hidden_dim, activation='relu')
        self.fc2 = tf.keras.layers.Dense(1)

    def call(self, x):
        x = self.fc1(x)
        return self.fc2(x)

In [9]:
# WGAN parameters
latent_dim = 16
hidden_dim = 32
input_dim = node_features.shape[1]

In [10]:
generator = WGANGenerator(latent_dim, hidden_dim, input_dim)
discriminator = WGANDiscriminator(input_dim, hidden_dim)

In [11]:
# Optimizers
g_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
d_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

In [12]:
# Step 3: WGAN Training for Class Balancing
batch_size = 8
num_epochs = 10

In [13]:
real_data = node_features[labels == 1]  # Fraudulent data (minority class)
target_minority_class = np.sum(labels == 0)  # Non-fraudulent (majority class)

In [14]:
@tf.function
def train_step(real_data):
    for _ in range(5):  # Update discriminator 5 times for every generator update
        z = tf.random.normal([real_data.shape[0], latent_dim])
        with tf.GradientTape() as d_tape:
            fake_data = generator(z, training=True)
            d_real = discriminator(real_data, training=True)
            d_fake = discriminator(fake_data, training=True)
            d_loss = -tf.reduce_mean(d_real) + tf.reduce_mean(d_fake)

        d_gradients = d_tape.gradient(d_loss, discriminator.trainable_variables)
        d_optimizer.apply_gradients(zip(d_gradients, discriminator.trainable_variables))

    with tf.GradientTape() as g_tape:
        z = tf.random.normal([real_data.shape[0], latent_dim])
        fake_data = generator(z, training=True)
        g_loss = -tf.reduce_mean(discriminator(fake_data, training=True))

    g_gradients = g_tape.gradient(g_loss, generator.trainable_variables)
    g_optimizer.apply_gradients(zip(g_gradients, generator.trainable_variables))

    return d_loss, g_loss

In [15]:
for epoch in range(num_epochs):
    current_minority_count = np.sum(labels == 1)
    if current_minority_count >= target_minority_class:
        break

    d_loss, g_loss = train_step(real_data)

    # Update labels and node_features
    z = tf.random.normal([real_data.shape[0], latent_dim])
    generated_samples = generator(z)
    labels = np.concatenate([labels, np.ones(generated_samples.shape[0], dtype=np.int64)])
    node_features = np.concatenate([node_features, generated_samples.numpy()])

    print(f'Epoch {epoch+1}/{num_epochs}, Loss D: {d_loss.numpy()}, Loss G: {g_loss.numpy()}')


Epoch 1/10, Loss D: 63.382938385009766, Loss G: 0.0034128082916140556
Epoch 2/10, Loss D: 61.935787200927734, Loss G: 0.010986598208546638
Epoch 3/10, Loss D: 60.50090789794922, Loss G: 0.01057228073477745
Epoch 4/10, Loss D: 59.06543731689453, Loss G: 0.003476534504443407
Epoch 5/10, Loss D: 57.61684799194336, Loss G: -0.0009680814109742641
Epoch 6/10, Loss D: 56.168880462646484, Loss G: -0.005858915857970715
Epoch 7/10, Loss D: 54.73793411254883, Loss G: -0.016547486186027527
Epoch 8/10, Loss D: 53.296348571777344, Loss G: -0.005269114393740892
Epoch 9/10, Loss D: 51.85076904296875, Loss G: -0.016116635873913765
Epoch 10/10, Loss D: 50.40438461303711, Loss G: -0.020673412829637527


In [16]:
# Step 4: GAT Layer using TensorFlow (custom implementation)
class GATLayer(tf.keras.layers.Layer):
    def __init__(self, output_dim, num_heads=1, dropout_rate=0.6):
        super(GATLayer, self).__init__()
        self.output_dim = output_dim
        self.num_heads = num_heads
        self.dropout_rate = dropout_rate

    def build(self, input_shape):
        self.W = self.add_weight(shape=(input_shape[-1], self.output_dim),
                                 initializer='glorot_uniform',
                                 trainable=True)
        self.attn_kernel = self.add_weight(shape=(2 * self.output_dim, 1),
                                           initializer='glorot_uniform',
                                           trainable=True)
        self.leaky_relu = tf.keras.layers.LeakyReLU(alpha=0.2)

    def call(self, node_features, adj_matrix, training=True):
        h = tf.matmul(node_features, self.W)
        num_nodes = h.shape[0]
        h_expanded = tf.tile(tf.expand_dims(h, axis=0), [num_nodes, 1, 1])
        h_expanded_transposed = tf.transpose(h_expanded, [1, 0, 2])

        a_input = tf.concat([h_expanded, h_expanded_transposed], axis=-1)
        e = self.leaky_relu(tf.matmul(a_input, self.attn_kernel))

        attention = tf.where(adj_matrix > 0, e, tf.zeros_like(e))
        attention = tf.nn.softmax(attention, axis=1)

        h_prime = tf.matmul(attention, h)
        if training:
            h_prime = tf.nn.dropout(h_prime, rate=self.dropout_rate)

        return tf.nn.elu(h_prime)

In [17]:
def create_adjacency_matrix(edge_index, num_nodes):
    adj_matrix = np.zeros((num_nodes, num_nodes), dtype=np.float32)
    for i, j in zip(edge_index[0], edge_index[1]):
        adj_matrix[i, j] = 1
    return adj_matrix

In [18]:
class GATModel(tf.keras.Model):
    def __init__(self, num_features, hidden_size):
        super(GATModel, self).__init__()
        self.gat_layer = GATLayer(output_dim=hidden_size)

    def call(self, node_features, adj_matrix, training=False):
        return self.gat_layer(node_features, adj_matrix, training)


In [19]:
# Step 5: Apply GAT on the reduced and balanced dataset
adj_matrix = create_adjacency_matrix(edge_index, num_nodes=len(node_features))
gat_model = GATModel(num_features=node_features.shape[1], hidden_size=128)
gat_output = gat_model(node_features, adj_matrix, training=True)

ResourceExhaustedError: OOM when allocating tensor with shape[274535,274535,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Tile]

In [ ]:
# Print the output of GAT
print("GAT output:", gat_output)